In [1]:
!pip install Keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.applications import densenet
from keras.models import Sequential, Model, load_model
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, Callback
from keras import regularizers
from keras import backend as K
import sys
from matplotlib import pyplot
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.optimizers import SGD
from keras.utils import multi_gpu_model
# import tensorflow as tf
# from keras.backend.tensorflow_backend import set_session
from keras.callbacks import ModelCheckpoint
import pandas as pd
from keras.layers.normalization import BatchNormalization
# from tensorflow.keras.callbacks import TensorBoard
import time
import keras

You should consider upgrading via the 'pip install --upgrade pip' command.


Using TensorFlow backend.


In [3]:
img_width, img_height = 224, 224
nb_train_samples = 539231
nb_validation_samples = 240424
epochs = 10
batch_size = 100
n_classes = 1

In [4]:
train_data_dir = "../../../../../raid/Data/Master_Dataset/mix/train"
validation_data_dir = "../../../../../raid/Data/Master_Dataset/mix/validation"

train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    fill_mode = 'constant',
    cval = 1,
    rotation_range = 5,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')



Found 539231 images belonging to 2 classes.
Found 240424 images belonging to 2 classes.


In [5]:
from keras.backend.tensorflow_backend import set_session
import tensorflow as tf
config = tf.ConfigProto()
config.gpu_options.allow_growth = True  # dynamically grow the memory used on the GPU
config.log_device_placement = True  # to log device placement (on which device the operation ran)
sess = tf.Session(config=config)
set_session(sess) 

In [6]:
base_model = densenet.DenseNet121(input_shape=(img_width, img_height, 3),
                                     weights="densenet121_weights_tf_dim_ordering_tf_kernels_notop.h5",
                                     include_top=False,
                                     pooling='avg')

for layer in base_model.layers:
    layer.trainable = True

x = base_model.output
x = Dense(n_classes, activation='sigmoid', name = 'prediction')(x)
model = Model(inputs = base_model.input, outputs = x)



Instructions for updating:
Colocations handled automatically by placer.


In [7]:
early_stop = EarlyStopping(monitor='val_loss', patience=3, verbose=1, mode='auto',min_delta=1e-2, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_accuracy', factor=0.2, patience=1, mode='auto',verbose=1, min_delta=1e-2, min_lr=0.00001)

In [8]:
i = 0
class LossHistory(Callback):
    def on_epoch_end(self, epoch, logs=None):
        global i
        self.model.save("paper/weights_of_epoch_" + str(i) + ".hdf5")
#         summarize_diagnostics(self.model_history, i)
        i += 1
        

In [9]:
e = 10
# model = tf.keras.models.load_model('/home/vision/mrunal/detect_deepfake/densenet121/paper/weights_of_epoch_0.hdf5')
sgd = SGD(lr=0.001, decay=1e-6, momentum=0.9, nesterov=True)
model = multi_gpu_model(model, gpus=2)
model.compile(optimizer=sgd, loss='binary_crossentropy', metrics=['accuracy'])
final_location = []
lh = LossHistory()
# checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
callbacks_list = [lh, early_stop, reduce_lr]

model_history = model.fit_generator(
train_generator,
epochs=e,
steps_per_epoch = nb_train_samples // train_generator.batch_size,
validation_data=validation_generator,
verbose=1,
validation_steps=nb_validation_samples // validation_generator.batch_size,
callbacks=callbacks_list)

#model.save("weights_best_" + str(e) + ".hdf5")

# _,acc=model.evaluate_generator(validation_generator, steps=nb_validation_samples, max_queue_size=10,verbose=1, workers=1, use_multiprocessing=False)
# print('> %.3f' % (acc * 100.0))

loss=model_history.history['loss']
acc=model_history.history['accuracy']
valacc=model_history.history['val_accuracy']
valloss=model_history.history['val_loss']
location = [e,loss,acc,valacc, valloss]
final_location.append(location)
save = pd.DataFrame(final_location,columns=['epochs','loss','acc','valacc','valloss'])
save.to_csv('/home/vision/mrunal/detect_deepfake/densenet121/paper/densenet_log_'+str(e)+'.csv')

Instructions for updating:
Use tf.cast instead.
Epoch 1/10
5392/5392 [==============================] - 7986s 1s/step - loss: 0.1242 - accuracy: 0.9480 - val_loss: 0.4775 - val_accuracy: 0.8698
Epoch 2/10
5392/5392 [==============================] - 7788s 1s/step - loss: 0.0488 - accuracy: 0.9807 - val_loss: 0.3897 - val_accuracy: 0.8513

Epoch 00002: ReduceLROnPlateau reducing learning rate to 0.00020000000949949026.
Epoch 3/10
5392/5392 [==============================] - 7153s 1s/step - loss: 0.0301 - accuracy: 0.9884 - val_loss: 0.3937 - val_accuracy: 0.8819
Epoch 4/10
5392/5392 [==============================] - 7007s 1s/step - loss: 0.0272 - accuracy: 0.9893 - val_loss: 0.4800 - val_accuracy: 0.8827

Epoch 00004: ReduceLROnPlateau reducing learning rate to 4.0000001899898055e-05.
Epoch 5/10
5392/5392 [==============================] - 6853s 1s/step - loss: 0.0249 - accuracy: 0.9903 - val_loss: 0.5188 - val_accuracy: 0.8909
Restoring model weights from the end of the best epoch

Ep

In [18]:
# PREDICTIONS TESTING
%%bash
pip install Keras

You should consider upgrading via the 'pip install --upgrade pip' command.


In [ ]:
import keras
model = keras.models.load_model("densenet121/paper/weights_of_epoch_1.hdf5")

In [20]:
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
lambda_1 (Lambda)               (None, 224, 224, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
lambda_2 (Lambda)               (None, 224, 224, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
model_1 (Model)                 (None, 1)            7038529     lambda_1[0][0]                   
                                                                 lambda_2[0][0]             

In [22]:
# ff benchmark
from keras.preprocessing.image  import load_img, img_to_array
from tqdm import tqdm
import os
import numpy as np
test_data_dir = "/home/vision/mrunal/faceforensics_benchmark_images"
final_predictions = []
# for classes in os.listdir(test_data_dir):
#     class_path = os.path.join(test_data_dir, classes)

for images in tqdm(os.listdir(test_data_dir)):
    file_path = os.path.join(test_data_dir, images)
    image = load_img(file_path, target_size = (224,224))
    array_image = img_to_array(image)
    array_image = array_image*(1./255)
    array_image = np.expand_dims(array_image, axis = 0)
    result = model.predict(array_image)
    final_predictions.append(result)

100%|██████████| 1000/1000 [00:44<00:00, 26.06it/s]
